# Queries for data processing for CI/ASU Decision Theater presentation

Import GEE api and json library. Note that `ee.initialize` will fail if the key for GEE has not yet been setup

In [6]:
import ee
import json
import io

service_account = 'gef-ldmp-server@gef-ld-toolbox.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'dt_key.json')
ee.Initialize(credentials)
#ee.Initialize()

Define a geojson as a string to use for testing

In [3]:
geojson_text = '''{"type": "Polygon", "coordinates": [ [ [ 36.898123159960932, -0.220252698724199 ], [ 37.40818204121706, 0.915787536800825 ], [ 38.613775760549743, 0.011592247301316 ], [ 38.219639352306366, -0.892603042198193 ], [ 36.898123159960932, -0.220252698724199 ] ] ] }'''

## Get statistics on areas degraded within a polygon

In [55]:
aoi = ee.Geometry(json.loads(geojson_text))

# Function to pull areas that are saved as properties within a feature class,
# convert them to percentages of the total area, and return as a dictionary
def fc_areas_to_pct_dict(fc, normalize=True):
    # Note that there may be multiple features
    out = []
    for p in [feature['properties'] for feature in fc.getInfo()['features']]:
        areas = {}
        if normalize:
            denominator = sum(p.values())
        else:
            denominator = 1
        out.append({key: value/denominator for key,value in p.iteritems()})
    return out

def get_degradation_areas(aoi):
    """Return a list of areas within each degradation class for all polygons in geojson"""
    "Returns area within each of the five degradation classes for each feature in a geojson"
    te_prod = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_lp7cl_globe_2001_2015_modis").remap([-32768,1,2,3,4,5,6,7],[-32768,-2,-2,0,0,0,2,2])

    fields = ["Area No Data", "Area Degraded", "Area Stable", "Area Improved"]

    # multiply pixel area by the area which experienced each of the five transitions --> output: area in ha
    stats = te_prod.eq([-32768, -2, 0, 2]).rename(fields).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
    
    return fc_areas_to_pct_dict(stats)

print(get_degradation_areas(aoi))

[{u'Area Improved': 0.13545880081222242, u'Area No Data': 0.0006406838566133825, u'Area Degraded': 0.5389454846121604, u'Area Stable': 0.3249550307190038}]


In [57]:
# polygon area in hectares
area = aoi.area().divide(10000).getInfo()
print area

###########################################################/
# s2_02: Number of people living inside the polygon in 2015
pop_cnt = ee.Image("CIESIN/GPWv4/unwpp-adjusted-population-count/2015");
population = pop_cnt.reduceRegion(reducer=ee.Reducer.sum(), geometry=aoi, scale=1000, maxPixels=1e9).getInfo()
print population

##########################################################/
# s2_03: Main livelihoods
liv = ee.FeatureCollection("users/geflanddegradation/toolbox_datasets/livelihoodzones")

livImage = liv.filter(ee.Filter.neq('lztype_num', None)).reduceToImage(properties=['lztype_num'], reducer=ee.Reducer.first())

fields = ["Agro-Forestry","Agro-Pastoral", "Arid","Crops - Floodzone","Crops - Irrigated","Crops - Rainfed","Fishery","Forest-Based","National Park","Other","Pastoral","Urban"]

# multiply pixel area by the area which experienced each of the five transitions --> output: area in ha
livelihoodareas = livImage.eq([1,2,3,4,5,6,7,8,9,10,11,12]).rename(fields).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum(),30)
print stats.getInfo()


51004527707.0
{u'population-count': 2347497.286870436}
{u'type': u'FeatureCollection', u'features': [{u'geometry': {u'type': u'Polygon', u'geodesic': True, u'coordinates': [[[36.89812315996093, -0.220252698724199], [37.40818204121706, 0.915787536800825], [38.61377576054974, 0.011592247301316], [38.219639352306366, -0.892603042198193], [36.89812315996093, -0.220252698724199]]]}, u'type': u'Feature', u'id': u'0', u'properties': {u'Forest-Based': 0.0, u'Fishery': 8418.681309866846, u'Pastoral': 251701.4419693742, u'Agro-Pastoral': 1124279.2945632995, u'Arid': 0.0, u'Agro-Forestry': 0.0, u'Crops - Rainfed': 640837.4351405775, u'Urban': 0.0, u'Crops - Floodzone': 0.0, u'Crops - Irrigated': 0.0, u'National Park': 0.0, u'Other': 0.0}}], u'columns': {u'system:index': u'String', u'Forest-Based': u'Float', u'Fishery': u'Float', u'Pastoral': u'Float', u'Agro-Pastoral': u'Float', u'Arid': u'Float', u'Agro-Forestry': u'Float', u'Crops - Rainfed': u'Float', u'Urban': u'Float', u'Crops - Floodzone': 

In [50]:
##########################################################/
# s3_01: SDG 15.3.1 degradation classes 

te_sdgi = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_sdg1531_gpg_globe_2001_2015_modis")
sdg_areas = te_sdgi.eq([-32768,-1,0,1]).rename(["nodata","degraded","stable","improving"]).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
print fc_areas_to_pct_dict(sdg_areas)

###########################################################/
# s3_02: Productivity degradation classes
te_prod = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_lp7cl_globe_2001_2015_modis").remap([-32768,1,2,3,4,5,6,7],[-32768,-1,-1,0,0,0,1,1])
prod_areas = te_prod.eq([-32768,-1,0,1]).rename(["nodata","decline","stable","improvement"]).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
print fc_areas_to_pct_dict(prod_areas)

###########################################################/
# s3_03: Land cover degradation classes
te_land = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_lc_traj_globe_2001-2001_to_2015")
lc_areas = te_land.select("lc_dg").eq([-1,0,1]).rename(["degradation","stable","improvement"]).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
print fc_areas_to_pct_dict(lc_areas)

###########################################################/
# s3_04: soc degradation classes
te_socc_deg = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_soc_globe_2001-2015_deg").select("soc_deg")
soc_areas = te_socc_deg.eq([-32768,-1,0,1]).rename(["no data","degradation","stable","improvement"]).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
print fc_areas_to_pct_dict(soc_areas)

[{u'improving': 0.13732207194709742, u'degraded': 0.5403488674535836, u'nodata': 0.0011595520415940092, u'stable': 0.3211695085577249}]
[{u'decline': 0.5389454846121603, u'nodata': 0.0006406838566133824, u'improvement': 0.13545880081222242, u'stable': 0.3249550307190038}]
[{u'degradation': 0.0015229421138970199, u'improvement': 0.01210907421085375, u'stable': 0.9863679836752491}]
[{u'no data': 0.0010461909413170469, u'degradation': 0.004569578369692355, u'improvement': 0.003875883604160064, u'stable': 0.9905083470848306}]


In [52]:
###########################################################/
# s3_05: compute land cover classes for 2001 and 2015, and the transitions which occured
te_land = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_lc_traj_globe_2001-2001_to_2015").select("lc_tr")
te_prod = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_lp7cl_globe_2001_2015_modis").remap([-32768,1,2,3,4,5,6,7],[-32768,-1,-1,0,0,0,1,1])

fields = ["nodata","decline","stable","improvement"]

prod_forests = te_prod.updateMask(te_land.eq(11)).eq([-32768,-1,0,1]).rename(fields).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
print fc_areas_to_pct_dict(prod_forests)

prod_grasslands = te_prod.updateMask(te_land.eq(22)).eq([-32768,-1,0,1]).rename(fields).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
print fc_areas_to_pct_dict(prod_grasslands)

prod_agriculture = te_prod.updateMask(te_land.eq(33)).eq([-32768,-1,0,1]).rename(fields).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
print fc_areas_to_pct_dict(prod_agriculture)


[{u'decline': 0.44106435556403983, u'nodata': 7.367366703563312e-05, u'improvement': 0.19284696816349126, u'stable': 0.36601500260543324}]
[{u'decline': 0.5529370548327694, u'nodata': 0.00015125333157003256, u'improvement': 0.09677108201358707, u'stable': 0.35014060982207335}]
[{u'decline': 0.5695099043142127, u'nodata': 0.00020328707781605126, u'improvement': 0.13796717435782577, u'stable': 0.29231963425014545}]


In [56]:
###########################################################/
# s3_06: compute land cover classes for 2001 and 2015, and the transitions which occured
te_land = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_lc_traj_globe_2001-2001_to_2015")

# field names for annual land covers
fields = ["forest","grassland","agriculture","wetlands","artificial","other land-bare","water"]

# multiply pixel area by the area which experienced each of the lc classes --> output: area in ha
stats_bl = te_land.select("lc_bl").eq([1,2,3,4,5,6,7]).rename(fields).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
# multiply pixel area by the area which experienced each of the lc classes --> output: area in ha
stats_tg = te_land.select("lc_tg").eq([1,2,3,4,5,6,7]).rename(fields).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())

print fc_areas_to_pct_dict(stats_bl)
print fc_areas_to_pct_dict(stats_tg)

# field names for land cover transitions between 2001-2015
fields = ["for-for","for-gra","for-agr","for-wet","for-art","for-oth","for-wat",
          "gra-for","gra-gra","gra-agr","gra-wet","gra-art","gra-oth","gra-wat",
          "agr-for","agr-gra","agr-agr","agr-wet","agr-art","agr-oth","agr-wat",
          "wet-for","wet-gra","wet-agr","wet-wet","wet-art","wet-oth","wet-wat",
          "art-for","art-gra","art-agr","art-wet","art-art","art-oth","art-wat",
          "oth-for","oth-gra","oth-agr","oth-wet","oth-art","oth-oth","oth-wat",
          "wat-for","wat-gra","wat-agr","wat-wet","wat-art","wat-oth","wat-wat"]

# multiply pixel area by the area which experienced each of the lc transition classes --> output: area in ha
stats_ch = te_land.select("lc_tr").eq([11,12,13,14,15,16,17,21,22,23,24,25,26,26,31,32,33,34,35,36,37,41,42,43,44,45,46,47,51,52,53,54,55,56,57,
              61,62,63,64,65,66,67,71,72,73,74,75,76,77]).rename(fields).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
print fc_areas_to_pct_dict(stats_ch, normalize=False)


[{u'wetlands': 0.0008490522684556844, u'artificial': 1.3243479362564002e-05, u'water': 0.0013331282986892088, u'forest': 0.20891647871751073, u'grassland': 0.31252693899966477, u'agriculture': 0.47155693434852475, u'other land-bare': 0.004804223887792304}]
[{u'wetlands': 0.000729860772884661, u'artificial': 0.00015450739266000438, u'water': 0.0013331282986892086, u'forest': 0.21599691344427693, u'grassland': 0.30558720986503685, u'agriculture': 0.4713897419153938, u'other land-bare': 0.004808638311058581}]
[{u'for-oth': 0.0, u'art-wat': 0.0, u'wat-for': 0.0, u'oth-art': 0.0, u'for-wet': 26.8167085647583, u'agr-oth': 8.939597129821777, u'wet-wat': 0.0, u'for-gra': 1850.4208908081055, u'oth-oth': 9728.977827390034, u'gra-oth': 0.0, u'oth-gra': 0.0, u'oth-wat': 0.0, u'for-wat': 0.0, u'for-art': 0.0, u'agr-for': 8135.220675606821, u'wet-wet': 1451.2158399432315, u'agr-art': 143.0359592437744, u'gra-for': 8152.039930066874, u'gra-gra': 616364.9029626813, u'wat-agr': 0.0, u'oth-wet': 0.0, u'

In [ ]:
with io.open('out.json', 'w', encoding='utf-8') as f:
  f.write(json.dumps(d, ensure_ascii=False, indent=4))